In [3]:
import sys
sys.path.append('/Users/alecranjitkar/anaconda3/fa2_modified/fa2_modified')
print(sys.path)


['/Users/alecranjitkar/Desktop/DTU/Master/1Semester/social_graphs-1/Alec-jawa/week5', '/Users/alecranjitkar/anaconda3/lib/python311.zip', '/Users/alecranjitkar/anaconda3/lib/python3.11', '/Users/alecranjitkar/anaconda3/lib/python3.11/lib-dynload', '', '/Users/alecranjitkar/anaconda3/lib/python3.11/site-packages', '/Users/alecranjitkar/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/alecranjitkar/anaconda3/fa2_modified', '/Users/alecranjitkar/anaconda3/fa2_modified/fa2_modified', '/Users/alecranjitkar/anaconda3/fa2_modified/fa2_modified']
